In [1]:
import torch as t
from torch import nn
from torch.autograd import Variable
from torch.optim import Adam
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import CIFAR10
import numpy as np
from torch  import optim
import torchvision.utils as vutil
from tensorboard_logger import Logger
import torchvision as tv
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import default_collate
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from tensorboard_logger import configure, log_value, Logger
test_logger = Logger('tensorboard_log/runs/test_1')
# configure("tensorboard_log/runs/test")

In [2]:
class Config:
    lr=0.0002
    nz=100# 噪声维度
    image_size=64
    image_size2=64
    nc=3# 图片三通道
    ngf=64 #生成图片
    ndf=64 #判别图片
    gpuids=2
    beta1=0.5
    batch_size=1024
    max_epoch=12# =1 when debug
    workers=2
    clamp_num=0.01# WGAN 截断大小
    
opt=Config()

In [3]:
def my_collate(batch):
    batch = filter (lambda x:x is not None, batch)
    return default_collate(batch)
class MyImageFolder(ImageFolder):
    __init__ = ImageFolder.__init__
    def __getitem__(self, index):
        try: 
            return super(MyImageFolder, self).__getitem__(index), self.imgs[index]
        except Exception as e:
            print e

val_dataset = MyImageFolder('/home/x/data/pre/test_new/nouse',
                transform=transforms.Compose(\
                                             
                                             [transforms.Scale(opt.image_size),
                                              transforms.RandomCrop(opt.image_size) ,
#                                               transforms.RandomSizedCrop()
                                              transforms.ToTensor(),
                                              transforms.Normalize([0.5]*3,[0.5]*3)
                                             ]))

val_dataloader=t.utils.data.DataLoader(val_dataset,64,True,num_workers=opt.workers, collate_fn=my_collate)

In [4]:
class ModelD(nn.Module):
    def __init__(self,ngpu):
        super(ModelD,self).__init__()
        self.ngpu=ngpu
        self.model=nn.Sequential()
        self.model.add_module('conv1',nn.Conv2d(opt.nc,opt.ndf,4,2,1,bias=False))
        self.model.add_module('relu1',nn.LeakyReLU(0.2,inplace=True))
        
        self.model.add_module('conv2',nn.Conv2d(opt.ndf,opt.ndf*2,4,2,1,bias=False))
        self.model.add_module('bnorm2',nn.BatchNorm2d(opt.ndf*2))
        self.model.add_module('relu2',nn.LeakyReLU(0.2,inplace=True))
        
        self.model.add_module('conv3',nn.Conv2d(opt.ndf*2,opt.ndf*4,4,2,1,bias=False))
        self.model.add_module('bnorm3',nn.BatchNorm2d(opt.ndf*4))
        self.model.add_module('relu3',nn.LeakyReLU(0.2,inplace=True))
        
        self.model.add_module('conv4',nn.Conv2d(opt.ndf*4,opt.ndf*8,4,2,1,bias=False))
        self.model.add_module('bnorm4',nn.BatchNorm2d(opt.ndf*8))
        self.model.add_module('relu4',nn.LeakyReLU(0.2,inplace=True))
        
        self.model.add_module('conv5',nn.Conv2d(opt.ndf*8,1,4,1,0,bias=False))
#         self.model.add_module('sigmoid',nn.Sigmoid())
    def forward(self,input):
        gpuids=None
        if self.ngpu:
            gpuids=range(self.ngpu)
        return nn.parallel.data_parallel(self.model,input, device_ids=gpuids)
netd=ModelD(opt.gpuids)

In [5]:
class FinetuneModel(nn.Module):
    def __init__(self, pretrained_model, ngpu=opt.gpuids):
        self.ngpu = ngpu
        super(FinetuneModel,self).__init__()
        self.features = pretrained_model
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 4 * 4, 2048),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(2048, 2048),
            nn.ReLU(True),
            nn.Linear(2048, 2))      
        
    def forward(self, x):
        gpuids = None
        if self.ngpu:
            gpuids = range(self.ngpu)
        features =  self.features(x).view(-1,512*4*4)#, x, device_ids=gpuids,ou)
        return self.classifier(features)

In [6]:
if hasattr(netd.model,'conv5'):del netd.model.conv5
model = FinetuneModel(netd)

In [7]:
model.load_state_dict(t.load('class_current_best_model_94.4813829787.nph'))
model.cuda()

FinetuneModel (
  (features): ModelD (
    (model): Sequential (
      (conv1): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (relu1): LeakyReLU (0.2, inplace)
      (conv2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (bnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      (relu2): LeakyReLU (0.2, inplace)
      (conv3): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (bnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu3): LeakyReLU (0.2, inplace)
      (conv4): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (bnorm4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
      (relu4): LeakyReLU (0.2, inplace)
    )
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (8192 -> 2048)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (2048 -> 2048)
    (5)

In [8]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
      
        
def accuracy(output, target, topk=(1,)):
    """
    Computes the precision@k for the specified values of k
    @param output: score
    @param target: label
    
    """
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def val(model): 
    model.eval()
    acc_meter = AverageMeter()
    loss_meter = AverageMeter()
    for ii,data in enumerate(val_dataloader):
        input,label=data 
        val_input = Variable(input, volatile = True).cuda()
        val_label = Variable(label.type(t.LongTensor), volatile = True).cuda()
        score = model(val_input)
        acc = accuracy(score.cpu().data,label)[0][0]
        val_loss = t.nn.functional.cross_entropy(score,val_label)
        n = input.size()[0]
        acc_meter.update(acc,n)
        loss_meter.update(val_loss.data,n)
        if acc_meter.count>3000:break
    
#     log_value('loss_val', val_loss)
#     log_value('accuracy', acc)
    model.train()
    return acc_meter, loss_meter

def get_wrong(scores,label,paths):
    _, pred = scores.max(1)
    results = (pred == label).type_as(pred)
    wrong_imgs = [img for (img,result) in zip(paths, results) if result[0]<1 ]
    return wrong_imgs
    
    
def val_and_show(model): 
    model.eval()
    acc_meter = AverageMeter()
    loss_meter = AverageMeter()
    wrong_pics=[]
    for ii,data in enumerate(val_dataloader):
        (input,label),(path,_) = data
        
        val_input = Variable(input, volatile = True).cuda()
        val_label = Variable(label.type(t.LongTensor), volatile = True).cuda()
        score = model(val_input)
        acc = accuracy(score.cpu().data,label)[0][0]
        wrong_imgs = get_wrong(score.cpu().data,label,path)
        wrong_pics+=wrong_imgs
        val_loss = t.nn.functional.cross_entropy(score,val_label)
        n = input.size()[0]
        acc_meter.update(acc,n)
        loss_meter.update(val_loss.data,n)

    
#     log_value('loss_val', val_loss)
#     log_value('accuracy', acc)
    model.train()
    return acc_meter, loss_meter,wrong_pics

In [9]:
a,b,c=val_and_show(model)

/usr/local/lib/python2.7/dist-packages/PIL/Image.py:874: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:874: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [11]:
import json
json.dump(c,open('new_wrong3','w'))

In [12]:
c[0]

'0/g100702.jpg'

In [10]:
a.avg

93.00389894520698